<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/rule_ml_pair_pron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
pip install pymorphy2[fast]

## **Импорт**

In [151]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [152]:
from IPython.display import SVG, display, HTML

In [153]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Drive already mounted at ./content; to attempt to forcibly remount, call drive.mount("./content", force_remount=True).


In [154]:
import spacy
import numpy as np

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [155]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [156]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

annotations.csv       hv-info_gepatit2.txt	 sample_data
annotation_table.csv  neuroleptic_antidepr2.txt
content		      rakpobedim_ru2.txt
.shortcut-targets-by-id
MyDrive
.file-revisions-by-id
.Trash


In [157]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

## **Обработка данных**

In [158]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-25 14:13:48--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt.1’

neuroleptic_antidep 100%[===================>]  11.12M  48.4MB/s    in 0.2s    

2021-05-25 14:13:49 (48.4 MB/s) - ‘neuroleptic_antidepr2.txt.1’ saved [11655409/11655409]

--2021-05-25 14:13:49--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:

In [159]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-25 14:13:50--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-25 14:13:50 ERROR 404: Not Found.

--2021-05-25 14:13:50--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100469 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv.1’

annotation_table.cs 100%[===================>]   2.00M  --.-KB/s    in 0.

In [160]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [161]:
posts = my_txt.split('----')

In [162]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [163]:
morph.parse('он')[0].normal_form

'он'

## **Обучение векторизатора**

In [164]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]


sentences_tokenized = [tokenizer.tokenize(morph.parse(i)[0].normal_form) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [165]:
count = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      count +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 19:
        new_row = new_row.split('\t')[:20]
        new_row[-1] = new_row[-1][:-1] # cut \n
      writer.writerow(new_row)
count

27019

In [166]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [167]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
!ls
# !rm annotation_table.csv.2 annotation_table.csv.1 annotation_table.csv annotations.csv

annotations.csv		hv-info_gepatit2.txt	     rakpobedim_ru2.txt
annotation_table.csv	hv-info_gepatit2.txt.1	     rakpobedim_ru2.txt.1
annotation_table.csv.1	neuroleptic_antidepr2.txt    sample_data
content			neuroleptic_antidepr2.txt.1


In [168]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,2385.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,0.0,NaN
27014,таможенные,0,22,68.0,0.0,2386.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,0.0,plur
27015,пошлины,0,22,68.0,0.0,2387.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,0.0,plur
27016,как,0,22,68.0,0.0,2388.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,0.0,NaN


In [169]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [170]:
# tok_list = []
# for sent in 
# for token in anns_df.HEAD:
#   try:
#     tok_list.append(ft.wv.__getitem__(token.lower()))
#   except KeyError:
#     tok_list.append(0)
#     print(token, 'here')
# anns_df['TOKEN_VECT'] = tok_list

In [171]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [172]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [173]:
anns_df['SENT_VECT'] = sent_vect_list

Проверим данные

In [174]:
anns_df.SENT_NUM[18]

1.0

In [175]:
# topics = list(set(df.TOPIC_NUM.values)) 
# for topic_num in topics:
#   # if topic_num == 2:
#   #   break
#   # извлекаем предложения топика
#   sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
#   for sent_num in sents_in_topic:
#     sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num)]

In [176]:
# vvv = 0
# with open('content/MyDrive/coref_fasttext60k_1.csv') as csvfile:
#   with open('new_coref_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     for new_row in csvfile:
#       vvv +=1
#       # print(i.split('\t'))
#       if len(new_row.split('\t')) > 9:
#         new_row = new_row.split('\t')[:9]
#       writer.writerow(new_row)
# vvv

In [177]:
# cor_df = pd.read_csv('content/MyDrive/coref_fasttext60k_1.csv', delimiter=' ')
# cor_df

In [178]:
# cosine_similarity([np.matrix(cor_df.TOKEN_VECT[2]).A[0]], [np.matrix(cor_df.TOKEN_VECT[3]).A[0]])[-1][-1]

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

In [179]:
# cor_df.TOKEN_VECT = [np.matrix(i).A[0] for i in cor_df.TOKEN_VECT]
# cor_df.HEAD_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_VECT]
# cor_df.SENT_VECT = [np.matrix(i).A[0] for i in cor_df.SENT_VECT]
# cor_df.HEAD_CHILDS_VECT = [np.matrix(i).A[0] for i in cor_df.HEAD_CHILDS_VECT]


Добавим размеченные данные, и посмотрим, что получилось.

In [180]:
# cor_df['NER'] = anns_df['NER']
# cor_df['COREFERENCE_CLUSTER'] = anns_df['COREFERENCE_CLUSTER']
# cor_df['ANN_TOKEN'] = anns_df['TOKEN']
# cor_df

In [181]:
# cor_df.loc()

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [182]:
# df = anns_df.iloc[:15000,:]
df = anns_df.iloc[:17200,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [183]:
len(df.loc[df['NER'] == 1])

604

Из них местоимений (прономинальные анафоры).

In [184]:
len(df.loc[df['NER'] == 3])

117

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [185]:
len(df.loc[df['NER'] == 2])

230

In [186]:
len(df.loc[df['NER'].isin([2,3])])

347

In [187]:
len(df.loc[df['NER'].isin([1,2,3])])

951

In [188]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [189]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.6232421, 0.9337824, 0.9163523, 0.037306197,...","[-1.1185107, 1.2558647, 0.4116446, 0.3824252, ...","[0.031031283, 0.40716556, 0.10485903, -0.32340..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,"[0.19384387, 0.9841908, 0.59680647, -0.1232337...","[0.6232421, 0.9337824, 0.9163523, 0.037306197,...","[0.031031283, 0.40716556, 0.10485903, -0.32340..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,"[0.2724266, 0.8496873, 0.5323043, -0.006485186...","[0.19384387, 0.9841908, 0.59680647, -0.1232337...","[0.031031283, 0.40716556, 0.10485903, -0.32340..."
21,лекарством,0,1,1.0,1.0,21.0,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,2,0.0,sing,"[-0.86026186, 0.9518488, 0.2662522, 0.6086677,...","[0.497284, -0.033726256, -0.096474834, -0.1449...","[0.031031283, 0.40716556, 0.10485903, -0.32340..."
142,рисполепта,0,1,1.0,13.0,142.0,inan,nomn,femn,NaN,NOUN,nmod,дозировку,inan,accs,femn,NOUN,1,142.0,sing,"[0.16099995, 0.6118273, 0.7791424, -0.20835473...","[-0.06530798, 0.74100107, 0.85917133, -0.05981...","[0.031031283, 0.40716556, 0.10485903, -0.32340..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16961,капецитабина,0,20,0.0,23.0,372.0,anim,gent,masc,NaN,NOUN,obj,назначил,NaN,NaN,masc,VERB,1,372.0,sing,"[0.48648843, 0.37744576, 0.8625318, -0.2592974...","[-0.22197202, 1.3923193, 0.21430084, 0.1339727...","[0.09802531, 0.3569541, 0.4447292, -0.17905927..."
17147,гемцитабина,0,20,3.0,1.0,558.0,anim,gent,masc,NaN,NOUN,nmod,побочку,inan,accs,femn,NOUN,1,558.0,sing,"[0.033902973, 0.74568355, 0.57660097, -0.13914...","[0.3405775, 0.5953728, 0.3134873, 0.3838977, 0...","[0.3292228, 0.63914156, 1.2842541, -0.483854, ..."
17164,капецитабине,0,20,3.0,2.0,575.0,anim,loct,masc,NaN,NOUN,nmod,отзывов,inan,gent,masc,NOUN,1,575.0,sing,"[0.45582873, 0.3345258, 0.41224733, 0.06375323...","[-0.055988405, 0.26380706, 0.13064781, 1.23698...","[0.50670713, 0.5101145, 0.33710495, 0.03292200..."
17182,гемцитабин,1,20,4.0,1.0,593.0,anim,nomn,masc,NaN,NOUN,nmod,тему,inan,accs,femn,NOUN,1,593.0,sing,"[-0.030403014, 0.95414394, 0.33714616, 0.01499...","[0.6892271, 1.0605696, -0.9705763, -0.13760354...","[0.09989464, 0.43444163, 0.1264321, -0.1312554..."


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [190]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.6232421, 0.9337824, 0.9163523, 0.037306197,...","[-1.1185107, 1.2558647, 0.4116446, 0.3824252, ...","[0.031031283, 0.40716556, 0.10485903, -0.32340..."
576,зелдокс,0,2,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.28275588, 0.9074949, 0.62305677, 0.00133830...","[-1.1185107, 1.2558647, 0.4116446, 0.3824252, ...","[0.12899838, 0.44297773, 0.09021814, -0.071725..."
1870,оланзапин,0,3,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.25220567, 1.1273324, 0.6811144, -0.46164748...","[-1.1185107, 1.2558647, 0.4116446, 0.3824252, ...","[0.059913795, 0.69394726, 0.042001426, -0.2056..."
5555,галоперидол,0,4,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,1,2.0,sing,"[-0.020051096, 1.0542554, 0.44067252, -0.21563...","[-0.5705146, 0.8037205, -0.49532524, -0.399111...","[0.12010475, 0.36699748, 0.1557671, -0.2441451..."
6200,тиаприд,0,5,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,3.0,sing,"[0.26712707, 0.81755257, 0.8784663, -0.1391943...","[-1.1185107, 1.2558647, 0.4116446, 0.3824252, ...","[0.12010475, 0.36699748, 0.1557671, -0.2441451..."
8450,сероквель,0,6,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.56048286, 0.54695064, 0.8539475, 0.07710416...","[-1.1185107, 1.2558647, 0.4116446, 0.3824252, ...","[0.13721968, 0.37743267, 0.10938035, -0.151037..."
8541,день,0,7,0.0,0.0,2.0,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,0,0.0,sing,"[-0.5870817, 0.18798962, -0.7799769, -0.027786...","[-0.5870817, 0.18798962, -0.7799769, -0.027786...","[0.13721968, 0.37743267, 0.10938035, -0.151037..."
11117,500,0,8,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0,0.0,NaN,"[-0.11592989, 0.24742971, 0.5241441, -0.141168...","[0.13797309, 0.02536428, 0.9554093, -0.3865424...","[0.0336666, 0.44736698, 0.20381391, -0.0340766..."
11787,капецитабин,0,10,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,1,2.0,sing,"[0.47605246, 0.54180163, 0.6562714, -0.1366656...","[0.47605246, 0.54180163, 0.6562714, -0.1366656...","[0.13815327, 0.5056665, 0.035681687, -0.113360..."
11806,иресса,0,11,0.0,0.0,2.0,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,1,2.0,NaN,"[0.26491448, 0.4622752, 0.5819945, -0.58622205...","[0.7603951, 0.2526962, 0.1358703, -0.8557593, ...","[0.13815327, 0.5056665, 0.035681687, -0.113360..."


In [191]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [192]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
# head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [193]:
df.TOKEN_VECT[710]

0

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [194]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [195]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
# make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [196]:
df.TOKEN[1]

'препарат'

In [197]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на обработанные данные.

In [198]:
df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,0.503382,0.524634,0.940147
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,sing,0.589472,0.524634,0.940147
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,0.753553,0.870663,0.940147
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,0.781835,0.580844,0.940147
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,0.746906,0.627247,0.940147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17195,теперь,1,20,4.0,1.0,606.0,NaN,NaN,NaN,NaN,ADVB,advmod,назначили,NaN,NaN,NaN,VERB,0,0.0,NaN,0.497563,0.535626,0.920162
17196,нам,1,20,4.0,1.0,607.0,NaN,datv,NaN,1per,NPRO,iobj,назначили,NaN,NaN,NaN,VERB,0,0.0,plur,0.530648,0.535626,0.920162
17197,назначили,1,20,4.0,1.0,608.0,NaN,NaN,NaN,NaN,VERB,conj,пошел,NaN,NaN,masc,VERB,0,0.0,plur,0.671222,0.562536,0.920162
17198,пить,1,20,4.0,1.0,609.0,NaN,NaN,NaN,NaN,INFN,xcomp,назначили,NaN,NaN,NaN,VERB,0,0.0,NaN,0.593482,0.535626,0.920162


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [199]:
# df.SENT_NUM = [int(i) for i in df['SENT_NUM']]
# df.POST_NUM = [int(i) for i in df['POST_NUM']]
# # df.TOKEN_NUM = [float(i) for i in df['TOKEN_NUM']]
df.TOPIC_NUM = [int(i) for i in df['TOPIC_NUM']]
df.IS_ANSWER = [int(i) for i in df['TOPIC_NUM']]
# df.NER = [int(i) for i in df['NER']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [200]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [201]:
df.loc[df['TOKEN_NUM'] == 344 ]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
344,рисполепта,1,1,2.0,4.0,344.0,inan,nomn,femn,NaN,NOUN,nmod,приеме,inan,loct,masc,NOUN,1,344.0,sing,0.752764,0.493349,0.988274
918,вышло,2,2,6.0,3.0,344.0,NaN,NaN,neut,NaN,VERB,ROOT,вышло,NaN,NaN,neut,VERB,0,0.0,sing,0.523590,0.489093,0.985874
2212,в,3,3,5.0,8.0,344.0,NaN,NaN,NaN,NaN,PREP,case,использовании,inan,loct,neut,NOUN,0,0.0,NaN,0.496220,0.550559,0.877150
5897,и,4,4,6.0,7.0,344.0,NaN,NaN,NaN,NaN,CONJ,cc,выписывались,NaN,NaN,NaN,VERB,0,0.0,NaN,0.466287,0.495304,0.980942
6542,в,5,5,17.0,0.0,344.0,NaN,NaN,NaN,NaN,PREP,case,мозгу,inan,loc2,masc,NOUN,0,0.0,NaN,0.496220,0.482363,0.980942
8883,их,7,7,0.0,32.0,344.0,NaN,accs,NaN,3per,NPRO,obj,принимать,NaN,NaN,NaN,INFN,0,0.0,plur,0.439573,0.488235,0.984169
11513,отвратно,9,9,0.0,39.0,344.0,NaN,NaN,NaN,NaN,ADVB,advmod,перенесла,NaN,NaN,femn,VERB,0,0.0,NaN,0.528829,0.525238,0.947767
12936,будет,15,15,0.0,31.0,344.0,NaN,NaN,NaN,3per,VERB,advcl,проходить,NaN,NaN,NaN,INFN,0,0.0,sing,0.476870,0.585020,0.933171
15278,ковид,16,16,9.0,0.0,344.0,inan,nomn,masc,NaN,NOUN,conj,наименование,inan,nomn,neut,NOUN,0,0.0,sing,0.596576,0.494664,0.765683
16465,сказал,18,18,7.0,2.0,344.0,NaN,NaN,masc,NaN,VERB,acl:relcl,парень,anim,nomn,masc,NOUN,0,0.0,sing,0.453016,0.502068,0.867435


In [202]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)]


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
5,не,1,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,0.453280,0.491755,0.940147
6,знаю,1,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.529921,0.491755,0.940147
7,стоило,1,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.501426,0.491755,0.940147
8,ли,1,1,1.0,0.0,8.0,NaN,NaN,NaN,NaN,CONJ,advmod,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.472805,0.480831,0.940147
9,мне,1,1,1.0,0.0,9.0,NaN,datv,NaN,1per,NPRO,iobj,стоило,NaN,NaN,neut,VERB,0,0.0,sing,0.508374,0.480831,0.940147
10,писать,1,1,1.0,0.0,10.0,NaN,NaN,NaN,NaN,INFN,csubj,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.502695,0.480831,0.940147


In [203]:
for i in df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)].values:
  print(i[-5])

0.0
0.0
0.0
0.0
0.0
0.0


In [204]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)].values[0]

array(['лекарством', 1, 1, 1.0, 1.0, 21.0, 'inan', 'ablt', 'neut', nan,
       'NOUN', 'conj', 'нет', nan, nan, nan, 'PRED', 2, 0.0, 'sing',
       0.4626462757587433, 0.5759973526000977, 0.9401474595069885],
      dtype=object)

In [205]:
# sents_in_topic 
# topic_num
# df = df.reset_index()

In [206]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [273]:
def rule_eval(my_X, my_y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(my_X)):
    if my_X[i] != 0:
      if my_X[i] == my_y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif my_X[i] == 0:
      if my_X[i] == my_y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')


In [274]:
df.loc[(df['POS'] == 5) &  (df['PERSON'] == 1)].iloc[30:40, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2875,он,3,3,15.0,2.0,1007.0,-1,0,0,1,5,12,489,-1,-1,-1,0,3,987.0,1,0.500966,0.518757,0.985869
2888,его,3,3,15.0,3.0,1020.0,-1,3,0,1,5,21,495,0,1,2,1,0,0.0,1,0.508885,0.653317,0.985869
3067,их,3,3,15.0,12.0,1199.0,-1,3,-1,1,5,1,571,-1,-1,-1,5,0,0.0,0,0.439573,0.658850,0.970353
3127,них,3,3,16.0,2.0,1259.0,-1,1,-1,1,5,10,588,-1,-1,-1,0,0,0.0,0,0.415070,0.415629,0.965396
3362,ее,3,3,18.0,7.0,1494.0,-1,3,1,1,5,1,446,0,0,1,1,3,1490.0,1,0.502987,0.492648,0.963051
3452,она,3,3,19.0,0.0,1584.0,-1,0,1,1,5,22,670,-1,-1,1,6,3,1582.0,1,0.511228,0.663323,0.902176
3626,его,3,3,21.0,4.0,1758.0,-1,3,0,1,5,1,726,-1,-1,-1,0,3,1750.0,1,0.508885,0.607603,0.950669
3705,его,3,3,22.0,3.0,1837.0,-1,3,0,1,5,11,741,0,1,1,1,0,0.0,1,0.508885,0.489424,0.969995
3901,его,3,3,23.0,6.0,2033.0,-1,3,0,1,5,11,792,0,1,2,1,3,2031.0,1,0.508885,0.658935,0.964477
3923,его,3,3,23.0,7.0,2055.0,-1,3,0,1,5,11,792,0,1,2,1,3,2053.0,1,0.508885,0.658935,0.964477


In [275]:
topics = list(set(df.TOPIC_NUM.values))
topics
# all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [279]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      # current_anaphors = sent.loc[(df['POS'] == 5) &  (df['PERSON'] == 1)]
      current_anaphors = sent.loc[(df['POS'] == 5) &  (df['NER'] > 1)]
      # current_antecedents = sent.loc[(sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            # в противном случае присваиваем нулевой класс
            ann_clusters.append(anfr[-5])
            # и класс, который определило правило(антецедента из предложения)
            my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)




[2.0, 540.0, 11.0, 11.0, 108.0, 182.0, 182.0, 182.0, 182.0, 658.0, 658.0, 829.0, 829.0, 829.0, 829.0, 0.0, 0.0, 0.0, 0.0, 2.0, 679.0, 0.0, 0.0, 0.0, 1483.0, 0.0, 1750.0, 2031.0, 2053.0, 0.0, 0.0, 0.0, 3125.0, 3270.0, 3619.0, 5.0, 114.0, 174.0, 365.0, 0.0, 527.0, 10.0, 27.0, 41.0, 41.0, 2.0, 135.0, 135.0, 218.0, 239.0, 256.0, 270.0, 270.0, 2.0, 2.0, 447.0, 521.0, 521.0, 521.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1455.0, 1516.0, 1609.0, 1657.0, 1722.0, 1770.0, 0.0, 1905.0, 1984.0, 0.0, 0.0, 2071.0, 0.0, 164.0, 0.0, 410.0, 534.0, 0.0, 0.0, 707.0, 707.0, 801.0, 0.0, 871.0, 881.0, 1350.0, 0.0, 0.0, 0.0, 122.0, 165.0, 263.0, 513.0, 556.0, 654.0, 813.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1867.0, 2138.0, 2295.0, 305.0, 0.0, 0.0, 0.0, 0.0]
---
[2.0, 536.0, 11.0, 11.0, 108.0, 182.0, 182.0, 182.0, 182.0, 658.0, 658.0, 829.0, 829.0, 829.0, 829.0, 183.0, 183.0, 183.0, 388.0, 446.0, 686.0, 778.0, 987.0, 987.0, 1490.0, 1582.0, 1750.0, 2031.0, 2053.0, 2344.0, 3.0, 3.0, 3125.0, 3270.0, 3619.0,

In [280]:
rule_eval(my_clusters, ann_clusters)

52  true positive
25  false positive
14  true negative
28  false negative
---------
0.6753246753246753  presicion
0.65  recall
0.6624203821656051  f-score


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [212]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize

['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS',
 'NUMBER']

In [213]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Index(['inan', 'anim'], dtype='object')


array([-1,  0,  1, ..., -1, -1,  1])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [214]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Index(['inan', 'anim'], dtype='object'),
 'CASE': Index(['nomn', 'gent', 'datv', 'accs', 'loct', 'ablt', 'loc2', 'voct', 'gen2'], dtype='object'),
 'DEPENDENCY': Index(['ROOT', 'obj', 'appos', 'parataxis', 'conj', 'advmod', 'ccomp', 'iobj',
        'csubj', 'case', 'obl', 'det', 'nsubj', 'cc', 'discourse', 'fixed',
        'mark', 'amod', 'advcl', 'xcomp', 'nummod', 'nmod', 'nsubj:pass',
        'nummod:gov', 'cop', 'acl:relcl', 'acl', 'aux', 'flat:foreign', 'punct',
        'obl:agent', 'flat', 'aux:pass', 'orphan', 'expl', 'csubj:pass'],
       dtype='object'),
 'GENDER': Index(['masc', 'femn', 'neut'], dtype='object'),
 'HEAD': Index(['обсуждаем', 'препарат', 'рисперидон', 'рисполепт', 'знаю', 'стоило',
        'меня', 'нет', 'проблем', 'лекарством',
        ...
        'удобнее', 'иллюстрации', 'поиске', 'темам', 'посмотреть', 'случай',
        'описала', 'побочку', 'гемцитабина', 'прошлась'],
       dtype='object', length=2898),
 'HEAD_ANIMACY': Index(['inan', 'anim', 

In [215]:

# labels_dict = {}
# test_labels_dict = {}

# for i in X.columns:
#   X[i], lbl = pd.factorize(X[i])
#   labels_dict[i] = lbl

# for i in X_test.columns:
#   X_test[i], lbl = pd.factorize(X_test[i])
#   test_labels_dict[i] = lbl 

# y = cor_df['COREFERENCE_CLUSTER']
# y, y_label = pd.factorize(y)

# y_test = cor_df_test['COREFERENCE_CLUSTER']
# y_test, y_test_label = pd.factorize(y_test)

In [216]:
df.loc[ (df['NER'] == 3.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
475,его,1,1,7.0,1.0,475.0,-1,3,0,1,5,11,199,0,2,2,1,3,2.0,1,0.508885,0.368002,0.982037
566,он,1,1,9.0,3.0,566.0,-1,0,0,1,5,12,231,-1,-1,0,0,3,536.0,1,0.500966,0.428329,0.982037
587,он,2,2,1.0,0.0,13.0,-1,0,0,1,5,12,235,-1,-1,0,0,3,11.0,1,0.500966,0.553178,0.982037
594,он,2,2,1.0,1.0,20.0,-1,0,0,1,5,12,242,-1,-1,-1,0,3,11.0,1,0.500966,0.599523,0.982037
736,него,2,2,3.0,5.0,162.0,-1,1,0,1,5,10,293,-1,-1,-1,0,3,108.0,1,0.525656,0.557898,0.982959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14906,него,15,15,19.0,3.0,2314.0,-1,1,0,1,5,21,2453,-1,-1,-1,15,3,2295.0,1,0.525656,0.521555,0.975723
14967,он,16,16,1.0,4.0,33.0,-1,0,0,1,5,12,2504,-1,-1,0,0,3,0.0,1,0.500966,0.612627,0.975723
16328,они,18,18,6.0,2.0,207.0,-1,0,-1,1,5,12,2507,-1,-1,-1,9,3,0.0,0,0.415776,0.547781,0.950713
16391,его,18,18,6.0,5.0,270.0,-1,3,0,1,5,1,1588,-1,-1,-1,5,3,0.0,1,0.508885,0.531915,0.937106


In [217]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,1,1,0.0,0.0,2.0,1,0,0,-1,1,2,1,0,0,0,1,1,2.0,1,0.753553,0.870663,0.940147
3,рисполепт,1,1,0.0,0.0,3.0,0,1,1,-1,1,3,2,1,0,0,1,1,3.0,0,0.781835,0.580844,0.940147
4,сперидан,1,1,0.0,0.0,4.0,-1,-1,0,-1,2,4,3,0,1,1,1,1,4.0,1,0.746906,0.627247,0.940147
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.462646,0.575997,0.940147
142,рисполепта,1,1,1.0,13.0,142.0,0,0,1,-1,1,21,69,0,2,1,1,1,142.0,1,0.752764,0.458480,0.940147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16961,капецитабина,20,20,0.0,23.0,372.0,1,1,0,-1,1,1,1405,-1,-1,0,0,1,372.0,1,0.724331,0.585249,0.904786
17147,гемцитабина,20,20,3.0,1.0,558.0,1,1,0,-1,1,21,2895,0,2,1,1,1,558.0,1,0.646392,0.557727,0.616869
17164,капецитабине,20,20,3.0,2.0,575.0,1,4,0,-1,1,21,2277,0,1,0,1,1,575.0,1,0.737523,0.398407,0.752519
17182,гемцитабин,20,20,4.0,1.0,593.0,1,0,0,-1,1,21,121,0,2,1,1,1,593.0,1,0.619885,0.477174,0.886083


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [218]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)

In [219]:
train = train.reset_index()

In [220]:
test = test.reset_index()

In [221]:
test.head(5)

,index,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,11565,наркотики,9,9,0.0,50.0,396.0,0,3,0,-1,1,18,2121,-1,0,-1,11,2,0.0,0,0.457495,0.417915,0.814365
1,11924,таб,11,11,7.0,0.0,120.0,-1,-1,-1,-1,-1,2,2190,-1,-1,-1,-1,2,116.0,-1,0.636353,0.588255,0.963718
2,454,препарат,1,1,7.0,0.0,454.0,0,0,0,-1,1,12,189,-1,-1,-1,0,2,2.0,1,0.589472,0.518877,0.982037
3,10053,химия,7,7,8.0,8.0,1514.0,0,0,1,-1,1,12,1849,-1,-1,-1,0,2,0.0,1,0.561228,0.556533,0.964330
4,2263,их,3,3,5.0,12.0,395.0,-1,3,-1,1,5,1,571,-1,-1,-1,5,3,388.0,0,0.439573,0.658850,0.960911


In [222]:
# test = test.iloc[:,:24].drop(['TOKEN', 'index'], axis=1)
# test = test.drop(['TOKEN', 'index'], axis=1)

In [223]:
# test.loc[test['NER'] != 0]

In [224]:
# ['TOPIC_NUM',	'POST_NUM',	'TOKEN_NUM',	'GENDER',	'PERSON', 'TOKEN', 'TOKEN_VECT', 'IS_ANSWER', 'SENT_NUM', 'HEAD_VECT', 'HEAD_CHILDS_VECT', 
#  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',	'CASE',	'POS',	'DEPENDENCY',	'HEAD',	'HEAD_POS']

In [225]:
# best set

# X, y, X_test, y_test = train.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',				'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
# train.COREFERENCE_CLUSTER, test.drop(['TOKEN',  'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', 'HEAD_CHILDS_VECT', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	'HEAD_GENDER', 'ANIMACY',			'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

Делаем наборы для обучения моделей.

In [226]:
X, y, X_test, y_test = train.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', \
                                   'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'index', 'HEAD_ANIMACY',	'HEAD_CASE',\
                                   'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
                                   
train.COREFERENCE_CLUSTER, test.drop(['TOKEN','GENDER', 'NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', \
                                      'PERSON', 'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',    'index', 'HEAD_ANIMACY',	'HEAD_CASE',	\
                                      'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], test.COREFERENCE_CLUSTER

In [227]:
# X, y, X_test, y_test = train.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], train.COREFERENCE_CLUSTER, test.drop(['TOKEN', 'index'], axis=1).iloc[:,:24], test.COREFERENCE_CLUSTER

In [228]:
X.NER = [float(i) for i in X.NER]
X_test.NER = [float(i) for i in X_test.NER]


In [229]:
y

0        0.0
1        0.0
2      156.0
3      135.0
4        0.0
       ...  
272      0.0
273    564.0
274     58.0
275    135.0
276    806.0
Name: COREFERENCE_CLUSTER, Length: 277, dtype: float64

Обучаем модели.

In [230]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.6428571428571429

In [231]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.6714285714285714

In [232]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.6285714285714286

In [233]:
regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)


1.0

In [234]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.4

In [235]:

knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.6571428571428571

In [236]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [237]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0 and test.iloc[:,22:23].iloc[[i]].values[0][0] in (2.0, 3.0):
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [238]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [239]:
# my_eval(list(naive_pred), list(y_test))

In [240]:
my_eval(list(pred_knn), list(y_test))

20  true positive
22  false positive
0  true negative
0  false negative
---------
0.47619047619047616  presicion
1.0  recall
0.6451612903225806  f-score


In [241]:
my_eval(list(pred_entropy), list(y_test))

21  true positive
23  false positive
0  true negative
0  false negative
---------
0.4772727272727273  presicion
1.0  recall
0.6461538461538462  f-score


In [242]:
my_eval(list(pred), list(y_test))

19  true positive
25  false positive
0  true negative
0  false negative
---------
0.4318181818181818  presicion
1.0  recall
0.6031746031746031  f-score


In [243]:
my_eval(list(pred_randf), list(y_test))

18  true positive
26  false positive
0  true negative
0  false negative
---------
0.4090909090909091  presicion
1.0  recall
0.5806451612903226  f-score


In [244]:
my_eval(list(pred_regr), list(y_test))

0  true positive
70  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [245]:
my_eval(list(pred_svm), list(y_test))

2  true positive
20  false positive
0  true negative
0  false negative
---------
0.09090909090909091  presicion
1.0  recall
0.16666666666666669  f-score


In [246]:
# df['TOPIC_NUM'] == 1
# df.loc[df['TOPIC_NUM'] == 1]

In [247]:
df.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', \
         'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'HEAD_ANIMACY',	'HEAD_CASE',	\
         'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18]

,TOKEN_NUM,NER,COREFERENCE_CLUSTER,TOKEN_VECT,SENT_VECT
0,0.0,0,0.0,0.503382,0.940147
1,1.0,0,0.0,0.589472,0.940147
2,2.0,1,2.0,0.753553,0.940147
3,3.0,1,3.0,0.781835,0.940147
4,4.0,1,4.0,0.746906,0.940147
...,...,...,...,...,...
17195,606.0,0,0.0,0.497563,0.920162
17196,607.0,0,0.0,0.530648,0.920162
17197,608.0,0,0.0,0.671222,0.920162
17198,609.0,0,0.0,0.593482,0.920162


In [248]:
len(df.loc[(df['POS'] == 5) & (df['NER'] > 1)].TOKEN)

119

In [249]:
df.loc[(df['COREFERENCE_CLUSTER'] != 0) & (df['NER'] > 1)]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
366,препарата,1,1,2.0,5.0,366.0,0,1,0,-1,1,21,154,0,1,1,1,2,344.0,1,0.518561,0.424643,0.988274
421,препарат,1,1,5.0,0.0,421.0,0,0,0,-1,1,0,1,0,0,0,1,2,2.0,1,0.589472,0.870663,0.988274
429,препарат,1,1,6.0,0.0,429.0,0,0,0,-1,1,12,179,-1,-1,-1,0,2,2.0,1,0.589472,0.558607,0.988274
446,последнему,1,1,6.0,2.0,446.0,-1,2,0,-1,10,7,184,-1,-1,-1,0,2,442.0,1,0.581406,0.537553,0.988274
454,препарат,1,1,7.0,0.0,454.0,0,0,0,-1,1,12,189,-1,-1,-1,0,2,2.0,1,0.589472,0.518877,0.982037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14427,него,15,15,14.0,2.0,1835.0,-1,1,0,1,5,10,195,-1,-1,-1,9,3,1827.0,1,0.525656,0.541278,0.778427
14440,его,15,15,14.0,2.0,1848.0,-1,3,0,1,5,1,1175,-1,-1,-1,0,3,1827.0,1,0.508885,0.553295,0.778427
14478,него,15,15,14.0,6.0,1886.0,-1,1,0,1,5,21,2453,-1,-1,-1,15,3,1867.0,1,0.525656,0.521555,0.709445
14733,него,15,15,17.0,8.0,2141.0,-1,1,0,1,5,10,2278,-1,-1,1,0,3,2138.0,1,0.525656,0.563533,0.642833


# **ПОПАРНЫЙ МЕТОД**

In [250]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [251]:
topics = list(set(df.TOPIC_NUM.values))
topics


pair_anaphors = []
pair_antecedents = []
relation_list = []
counter = 0
ants_with_anaph = 0
len_anapfors = 0
for topic_num in topics:

  # all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] > 1)]
  all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

  all_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] == 1)]
  
  len_anapfors += len(all_anaphors)
  for anaphor in all_anaphors.values:
    # чтобы примеры влезли в память, и модель не переобучилась на антипримерах,
    # возьмем один положительный и два отрицательных класса для каждого анафора
    counter = 0
    is_relation = 0
    for antecedent in all_antecedents.values:
      

      # print(anaphor[-5], antecedent[5])
      if anaphor[-5] == antecedent[5]:
        ants_with_anaph += 1
        is_relation = 1
        # такие срезы, чтобы убрать лишние данные, и в том числе разметку с ответами
        pair_anaphors.append(list(anaphor)[3:6] )# + list(anaphor)[17:18]\
                             # + list(anaphor)[19:23])
        
        pair_antecedents.append(list(antecedent)[3:6]) # + list(antecedent)[17:18]\
                             # + list(antecedent)[19:23])
        
        relation_list.append(is_relation)
      else:
        if counter != 1:

          pair_anaphors.append(list(anaphor)[3:6] )# + list(anaphor)[17:18]\
                              # + list(anaphor)[19:23])
          
          pair_antecedents.append(list(antecedent)[3:6]) # + list(antecedent)[17:18]\
                              # + list(antecedent)[19:23])
          
          relation_list.append(is_relation)
          counter = 1

  # counter += 1
print(len_anapfors)
print('---')
print(pair_anaphors[0][0], pair_antecedents[0][0], relation_list[0])
len(pair_anaphors), len(pair_antecedents), len(relation_list)

725
---
1.0 0.0 0


(766, 766, 766)

In [252]:
# можно посмотреть, что получилось
relation_list[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [253]:
pair_anaphors[0]

[1.0, 0.0, 9.0]

In [254]:
# проверим, третьим элементом должен быть номер токена
len([i for i in relation_list if i==1])

88

In [255]:
# все сходится. мы взяли колонки с номерами поста, предложения и токена,\
# колонку NER, число, и вектора
df.loc[df.TOKEN_NUM == 21.0].iloc[[0]]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.462646,0.575997,0.940147


In [256]:
cosine = [cosine_similarity([pair_anaphors[i]], [pair_antecedents[i]])[0][0] for i in range(len(pair_anaphors))]
# pair_df = pd.DataFrame(columns=['anaphs', 'ants', 'rel'])
pair_df = pd.DataFrame(columns=['cosine', 'rel'])

# pair_df['anaphs'] = pair_anaphors
# pair_df['ants'] = pair_antecedents
pair_df['cosine'] = cosine
pair_df['rel'] = relation_list

print(pair_df.loc[pair_df['cosine'] > 1])
print(pair_df.loc[pair_df['rel'] > 1])
pair_df

Empty DataFrame
Columns: [cosine, rel]
Index: []
Empty DataFrame
Columns: [cosine, rel]
Index: []


,cosine,rel
0,0.993884,0
1,0.993127,0
2,0.995440,0
3,0.996729,0
4,0.997029,0
...,...,...
761,0.986387,0
762,0.986379,0
763,0.986649,0
764,0.986646,0


In [257]:
pair_df.loc[pair_df.rel != 0]

,cosine,rel
26,0.999889,1
27,0.999889,1
33,0.999994,1
36,0.999904,1
38,0.997928,1
...,...,...
643,0.999996,1
651,0.999984,1
669,0.999999,1
684,1.000000,1


In [258]:
train, test = train_test_split(pair_df, test_size=0.3, shuffle=True)
train = train.reset_index()
test = test.reset_index()

In [259]:
X, y, X_test, y_test = train.iloc[:, 1:2], train.rel, test.iloc[:, 1:2], test.rel


In [260]:
train.iloc[:1, 1:2]

,cosine
0,0.999986


In [261]:
test.iloc[:1, 1:2]

,cosine
0,0.999728


In [262]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
230  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [263]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
16  true positive
9  false positive
197  true negative
8  false negative
---------
0.64  presicion
0.6666666666666666  recall
0.6530612244897959  f-score


In [264]:
dtclf = tree.DecisionTreeClassifier(max_depth=2000, )
dtclf.fit(X, y)


print('-----')
pred = dtclf.predict(X_test)
pair_eval(pred, y_test)


-----
15  true positive
15  false positive
191  true negative
9  false negative
---------
0.5  presicion
0.625  recall
0.5555555555555556  f-score


In [265]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
15  true positive
15  false positive
191  true negative
9  false negative
---------
0.5  presicion
0.625  recall
0.5555555555555556  f-score


In [266]:
clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
rule_eval(pred, y_test)

-----
15  true positive
15  false positive
191  true negative
9  false negative
---------
0.5  presicion
0.625  recall
0.5555555555555556  f-score


In [267]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [268]:
# clf = tree.DecisionTreeClassifier(max_depth=2000, )
# clf.fit(X, y)
# clf.score(X_test, y_test)

# **Извлечение номинативных анафоров**

In [269]:
df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,1,1,0.0,0.0,1.0,0,0,0,-1,1,1,0,-1,-1,-1,0,0,0.0,1,0.589472,0.524634,0.940147
16,проблем,1,1,1.0,1.0,16.0,0,1,1,-1,1,12,7,-1,-1,-1,3,0,0.0,0,0.537508,0.575997,0.940147
19,частности,1,1,1.0,1.0,19.0,0,4,1,-1,1,15,10,-1,-1,-1,4,0,0.0,1,0.554322,0.453951,0.940147
24,мере,1,1,1.0,2.0,24.0,0,2,1,-1,1,15,12,-1,-1,-1,4,0,0.0,1,0.585255,0.520091,0.940147
25,врач,1,1,1.0,2.0,25.0,1,0,0,-1,1,12,11,-1,-1,-1,0,0,0.0,1,0.623946,0.470146,0.940147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,дереала,2,2,4.0,2.0,239.0,0,1,0,-1,1,21,274,0,0,0,1,0,0.0,1,0.632742,0.482606,0.982959
849,месяц,2,2,5.0,1.0,275.0,0,3,0,-1,1,10,324,-1,-1,-1,5,0,0.0,1,0.580459,0.493649,0.985874
853,манию,2,2,5.0,1.0,279.0,0,3,1,-1,1,10,326,-1,-1,1,0,0,0.0,1,0.505896,0.522945,0.985874
856,мг,2,2,5.0,1.0,282.0,0,0,0,-1,1,10,326,-1,-1,1,0,0,0.0,1,0.576025,0.522945,0.985874


In [270]:
df.loc[(df['POS'] == 1) & (df['NER'] > 1)] 

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,0,5,2,-1,1,4,7,-1,-1,-1,3,2,0.0,1,0.462646,0.575997,0.940147
217,таблетки,1,1,1.0,21.0,217.0,0,3,1,-1,1,10,101,-1,-1,1,0,2,0.0,0,0.531873,0.670917,0.940147
295,таблетки,1,1,2.0,0.0,295.0,0,3,1,-1,1,10,101,-1,-1,1,0,2,0.0,0,0.531873,0.670917,0.988274
366,препарата,1,1,2.0,5.0,366.0,0,1,0,-1,1,21,154,0,1,1,1,2,344.0,1,0.518561,0.424643,0.988274
421,препарат,1,1,5.0,0.0,421.0,0,0,0,-1,1,0,1,0,0,0,1,2,2.0,1,0.589472,0.870663,0.988274
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15630,прививки,16,16,16.0,0.0,696.0,0,1,1,-1,1,1,2636,1,0,0,1,2,0.0,1,0.581378,0.640188,0.915981
15651,вакцина,16,16,16.0,0.0,717.0,0,0,1,-1,1,21,2642,0,0,2,1,2,0.0,1,0.556139,0.453727,0.880401
15692,химиотерапии,16,16,17.0,3.0,758.0,0,1,1,-1,1,21,142,1,0,0,1,2,0.0,1,0.581732,0.582392,0.960285
15751,химиотерапий,16,16,18.0,1.0,817.0,0,1,1,-1,1,21,2609,0,1,1,1,2,0.0,0,0.641569,0.632239,0.948283


In [271]:
df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :].append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)])

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,1,1,0.0,0.0,1.0,0,0,0,-1,1,1,0,-1,-1,-1,0,0,0.0,1,0.589472,0.524634,0.940147
16,проблем,1,1,1.0,1.0,16.0,0,1,1,-1,1,12,7,-1,-1,-1,3,0,0.0,0,0.537508,0.575997,0.940147
19,частности,1,1,1.0,1.0,19.0,0,4,1,-1,1,15,10,-1,-1,-1,4,0,0.0,1,0.554322,0.453951,0.940147
24,мере,1,1,1.0,2.0,24.0,0,2,1,-1,1,15,12,-1,-1,-1,4,0,0.0,1,0.585255,0.520091,0.940147
25,врач,1,1,1.0,2.0,25.0,1,0,0,-1,1,12,11,-1,-1,-1,0,0,0.0,1,0.623946,0.470146,0.940147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15630,прививки,16,16,16.0,0.0,696.0,0,1,1,-1,1,1,2636,1,0,0,1,2,0.0,1,0.581378,0.640188,0.915981
15651,вакцина,16,16,16.0,0.0,717.0,0,0,1,-1,1,21,2642,0,0,2,1,2,0.0,1,0.556139,0.453727,0.880401
15692,химиотерапии,16,16,17.0,3.0,758.0,0,1,1,-1,1,21,142,1,0,0,1,2,0.0,1,0.581732,0.582392,0.960285
15751,химиотерапий,16,16,18.0,1.0,817.0,0,1,1,-1,1,21,2609,0,1,1,1,2,0.0,0,0.641569,0.632239,0.948283


In [272]:
print(len(df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]))
df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]

19


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
446,последнему,1,1,6.0,2.0,446.0,-1,2,0,-1,10,7,184,-1,-1,-1,0,2,442.0,1,0.581406,0.537553,0.988274
3539,препарат.точнее,3,3,21.0,0.0,1671.0,-1,-1,-1,-1,13,0,691,-1,-1,-1,14,2,3.0,-1,0.624693,0.866605,0.802883
5222,новейший,3,3,38.0,5.0,3354.0,-1,0,0,-1,10,0,1085,-1,0,0,8,2,3350.0,1,0.625232,0.592682,0.980942
6857,тиапридал,5,5,23.0,6.0,659.0,-1,-1,0,-1,0,12,958,-1,-1,-1,0,2,659.0,1,0.705900,0.508962,0.980942
8834,это,7,7,0.0,25.0,295.0,-1,-1,-1,-1,3,12,941,-1,-1,-1,9,3,293.0,-1,0.478646,0.486115,0.984169
8928,хт,7,7,1.0,2.0,389.0,-1,-1,-1,-1,-1,21,1634,0,0,0,1,2,0.0,-1,0.531953,0.486494,0.984169
8987,хт,7,7,1.0,8.0,448.0,-1,-1,-1,-1,-1,21,1651,0,1,2,1,2,0.0,-1,0.531953,0.536071,0.984169
9086,хт,7,7,1.0,13.0,547.0,-1,-1,-1,-1,-1,10,229,-1,-1,-1,5,2,0.0,-1,0.531953,0.576199,0.984169
9497,ксф,7,7,1.0,57.0,958.0,-1,-1,-1,-1,-1,12,1791,-1,-1,-1,0,2,0.0,-1,0.495159,0.643928,0.981088
9935,хт,7,7,7.0,0.0,1396.0,-1,-1,-1,-1,-1,21,1634,0,0,0,1,2,0.0,-1,0.531953,0.486494,0.977734


In [298]:
med_df = df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :]\
            .append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)]).\
            loc[:,['TOKEN_VECT', 'HEAD_VECT', 'SENT_VECT', 'NER']]
            

In [299]:
med_df

,TOKEN_VECT,HEAD_VECT,SENT_VECT,NER
1,0.589472,0.524634,0.940147,0
16,0.537508,0.575997,0.940147,0
19,0.554322,0.453951,0.940147,0
24,0.585255,0.520091,0.940147,0
25,0.623946,0.470146,0.940147,0
...,...,...,...,...
15630,0.581378,0.640188,0.915981,2
15651,0.556139,0.453727,0.880401,2
15692,0.581732,0.582392,0.960285,2
15751,0.641569,0.632239,0.948283,2
